# Object Relational Mapping (ORM)
### Esempi di utilizzo con Flask-SQLAlchemy 

Oggi parliamo di ORM, ovvero della tecnologia che ci consente di collegare un database ad una applicazione, mantenendosi però distaccata dal mondo SQL o da un particolare DB o dialetto. 
In questo modo possiamo scrivere del codice che sia **db-agnostic**.
Questo significa che grazie all'utilizzo di uno strumento di Object Relational Mapping (di seguito orm), possiamo delegare ad uno strumento (in questo caso una libreria) il compito di interagire con il database, mentre all'interno dell'applicazione noi lavoreremo soltanto con degli *oggetti* 

Partiamo con la definizione "ufficiale" di orm:

>Un ORM (Object-Relational Mapping) è uno strumento software che facilita la conversione dei dati tra un sistema di database relazionale e un linguaggio di programmazione orientato agli oggetti. Un ORM consente agli sviluppatori di interagire con un database utilizzando le stesse convenzioni e tecniche che usano per manipolare gli oggetti nel loro codice. Questo riduce la necessità di scrivere query SQL manualmente. 
L'ORM funziona mappando le tabelle del database a classi nel codice e le righe della tabella a istanze di queste classi.

Ho sempre pensato che al mondo c'è chi ama le definizioni e chi ama fare. Per vostra fortuna appartengo al secondo gruppo, quindi ci affacceremo a questi argomenti con esempi pratici.

In questo notebook utilizzeremo l'ORM SQLAlchemy, in particolare nella sua versione che si integra con Flask. Questo consente una certa semplicità, e di concentrarci solo su quelllo che ci serve senza perderci in complicazioni applicative. Iniziamo installando le dipendenze necessarie

In [2]:
%pip install Flask Flask-SQLAlchemy Flask-Migrate

# se usi Jupyter usa questo:
#!pip install Flask Flask-SQLAlchemy Flask-Migrate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Creiamo un model

Un "model" in un ORM rappresenta una tabella nel database. Ogni classe definita come model rappresenta una tabella e i suoi attributi rappresentano le colonne della tabella.
Questo è il modo in cui un model viene creato. 

In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [2]:
class Utente(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    details = db.Column(db.Text, nullable=True)
    
    def __repr__(self):
        return f"ID: {self.id} | Username: {self.username}"


with app.app_context():
    db.create_all()

Vediamo cosa è successo nella cella qua sopra: la prima cella di codice serve ad avviare il servizio e per ora può essere ignorata. Andiamo a vedere cosa succede dopo:  
  
`class Utente(db.Model)`: Con questa riga sto creando una classe, ereditando le caratteristiche della superclasse db.Model. In poche parole sto dicendo a Python che questa classe sarà un model, e che eredita tutti i metodi e attributi della classe Model di SQLAlchemy  
  
`id = db.Column(db.Integer, primary_key=True)`: qua sto definendo il primo attributo del mio model. L'id sarà infatti una colonna del DB (db.Column), se non passato esplicitamente sarà **autoincrementale** (autoincrement=True), sarà un **intero** (db.Integer), e sarà la **primary key** della tabella (primary_key=True)  
  
`username = db.Column(db.String(80), unique=True, nullable=False)`: qua definiamo un altro attributo. Questo attributo sarà una stringa di lunghezza massima 80 (db.String(80)) [la lunghezza si può omettere, ma fissare una lunghezza massima è preferibile per ottimizzare il db]. Questo attributo deve essere **unico** (unique=True), e non può mai essere vuoto (nullable=False). Per l'attributo email valgono le stesse considerazioni.  
  
`details = db.Column(db.Text, nullable=True)`: definiamo qua un ultimo attributo. Questo può essere un testo anche lungo (db.Text), e può essere vuoto (nullable=True)

Il metodo `__repr__` serve per dire quale deve essere la rappresentazione dell'oggetto quando viene elencato o stampato con print. Nella prossima cella lo vedremo in azione.

Queste due righe:
`with app.app_context():`  
    `db.create_all()`    

Servono a propagare sul DB il nostro model. Questa parte verrà trattata in dettaglio più avanti nel notebook, per ora possiamo ignorarla.  
 Devo usare `with app.app_context():` per dire al motore di proseguire anche se siamo su un notebook e non su una vera applicazione. Questo espediente verrà molto usato nelle prossime celle, ma è indispensabile per il funzionamento del notebook.

## Come uso un model?  

Facciamo subito un esempio. Per creare un nuovo oggetto, sarà sufficiente istanziare un nuovo oggetto della classe Utente.

In [3]:
with app.app_context():
    
    nuovo_utente = Utente(username="MogliaL", email="lorenzo.moglia@tasgroup.eu") # non metto nessun detail, mentre l'id è generato automaticamentr
    db.session.add(nuovo_utente)
    db.session.commit()

Bene, abbiamo creato il nostro primo utente!  
Ora, come faccio a controllare quanti utenti ho attualmente nel mio db? Posso controllarlo facendo una query sul mio model, che mi restituisca tutti gli utenti, con la rappresentazione che ho definito all'interno del metodo `__repr__`.  
  
Per vedere tutti gli oggetti di un certo model, utilizzo il metodo `<model>.query.all()`.

In [4]:
with app.app_context():
    users = Utente.query.all()
    print(users) 

[ID: 1 | Username: MogliaL]


Eccolo qua! Però un'applicazione con un solo user è abbastanza triste. Ogni applicazione ha bisogno di un utente admin, di un moderatore, e di alcuni utenti base o premium. Creiamoli, insieme  !

In [5]:
with app.app_context():
    utente_admin = Utente(username="admin", email="admin@applicazione.tech", details="admin user - superuser")
    db.session.add(utente_admin)
    utente_mod = Utente(username="mod", email="mod@applicazione.tech", details="admin user - moderator")
    db.session.add(utente_mod)
    utente_user1 = Utente(username="Mario_Rossi", email="user1@applicazione.tech", details="premium user")
    db.session.add(utente_user1)
    utente_user2 = Utente(username="Pietro_Verdi", email="user2@applicazione.tech", details="premium user")
    db.session.add(utente_user2)
    utente_user3 = Utente(username="Bonaventura_Larda", email="user3@applicazione.tech") 
    db.session.add(utente_user3)
    # se voglio aggiungere altri utenti, posso farlo qua, all'interno della stessa sessione di connessione al db. Il commit va eseguito una sola volta per sessione, e scrive sul db tutte le operazioni avvenute nella stessa sessione
    db.session.commit()

Vediamo ora tutti gli utenti dell'applicazione:

In [12]:
with app.app_context():
    users = Utente.query.all()
    print(users) 

[ID: 1 | Username: MogliaL, ID: 2 | Username: admin, ID: 3 | Username: mod, ID: 4 | Username: Mario_Rossi, ID: 5 | Username: Pietro_Verdi, ID: 6 | Username: Bonaventura_Larda]


Diciamo che ora voglio caricare l'utente che risponde al nome di Mario Rossi.

In [13]:
with app.app_context():
    user = Utente.query.filter_by(username='Mario_Rossi').first()
    print(user)


ID: 4 | Username: Mario_Rossi


OK! Ora carichiamo tutti i premium users

In [14]:
with app.app_context():
    premium_users = Utente.query.filter_by(details='premium user').all()
    print(premium_users)

[ID: 4 | Username: Mario_Rossi, ID: 5 | Username: Pietro_Verdi]


Abbiamo visto che sia mod che admin hanno nei details "admin user - {ruolo}". Andiamo a fare una query che carica tutti gli utenti i cui details **contengano** la dicitura "admin user"

In [15]:
with app.app_context():
    admin_users = Utente.query.filter(Utente.details.like('%admin user%')).all()
    print(admin_users)

[ID: 2 | Username: admin, ID: 3 | Username: mod]


# work in progress...